# Create the SQLite database

In [2]:
import sqlite3

# Create a database connection
conn = sqlite3.connect('database.db')

# Create a cursor object
c = conn.cursor()

In [8]:
# Create the tables

c.execute("""CREATE TABLE IF NOT EXISTS Infrastructure (
                id INTEGER PRIMARY KEY,
                UWInfrastructure TEXT,
                Address TEXT
            );""")

c.execute("""CREATE TABLE IF NOT EXISTS Vehicles (
                id INTEGER PRIMARY KEY,
                Name TEXT,
                Description TEXT,
                Branch TEXT
            );""")

c.execute("""CREATE TABLE IF NOT EXISTS Employees (
                id INTEGER PRIMARY KEY,
                EmployeeNumber TEXT,
                HomeAddress TEXT,
                WorkAddress TEXT
            );""")

c.execute("""CREATE TABLE IF NOT EXISTS Telemetry (
                id INTEGER PRIMARY KEY,
                VehicleID INTEGER,
                DriverID INTEGER,
                DepartDate TEXT,
                StartTime TEXT,
                StopTime TEXT,
                Distance REAL,
                DrivingDuration INTEGER,
                IdlingDuration INTEGER,
                StopDuration INTEGER,
                Odometer REAL,
                MaxSpeed REAL,
                FromAddress TEXT,
                ToAddress TEXT
            );""")

# Commit the changes
conn.commit()

# Importing the data

In [9]:
import csv

## Infrastructure

In [14]:
with open('Infrastructure.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.DictReader(f)
    infrastructure = list(reader)

    for row in infrastructure:
        # print(row)
        c.execute("INSERT INTO Infrastructure VALUES (:id, :UWInfrastructure, :Address)", (row['id'], row['Unitywater Infrastructure'], row['Address']))

conn.commit()

## Vehicles

In [17]:
with open('Vehicles.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.DictReader(f)
    vehicles = list(reader)

    for row in vehicles:
        # print(row)
        c.execute("INSERT INTO Vehicles VALUES (:id, :Name, :Description, :Branch)", (row['id'], row['Vehicle Name'], row['Vehicle Description'], row['Branch']))

    conn.commit()

## Employees

In [19]:
with open('Employees.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.DictReader(f)
    employees = list(reader)

    for row in employees:
        # print(row)
        c.execute("INSERT INTO Employees VALUES (:id, :EmployeeNumber, :HomeAddr, :WorkLocation)", (row['id'], row['Employee Number'], row['Home Address'], row['Domiciled Work Location']))

conn.commit()

## Telemetry

In [26]:
# Clear Table
c.execute("DELETE FROM Telemetry")
conn.commit()

In [27]:
def converthmstos(string):
    if len(string.split(':')) != 3:
        return 0
    # Convert "HH:MM:SS" to seconds
    h, m, s = string.split(':')
    return int(h) * 3600 + int(m) * 60 + int(s)

with open('Telemetry.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.DictReader(f)
    telemetry = list(reader)

    for row in telemetry:
        # print(row)
        c.execute("INSERT INTO Telemetry VALUES (:id, :VehicleID, :DriverID, :DepartDate, :StartTime, :StopTime, :Distance, :DrivingDuration, :IdlingDuration, :StopDuration, :Odometer, :MaxSpeed, :FromAddress, :ToAddress)",
                  (row['id'], row['Vehicle Name'], row['Driver'], row['Depart Date'], row['Start Time'], row['Stop Time'], float(row['Distance']), converthmstos(row['Driving Duration']), converthmstos(row['Idling Duration']), converthmstos(row['Stop Duration']), float(row['Odometer']), float(row['Max Speed']), row['From Address'], row['To Address']))
        
    conn.commit()

# Addresses

In [31]:
# Create a table
c.execute("CREATE TABLE IF NOT EXISTS Addresses (Address TEXT, Longitude REAL, Latitude REAL)")
conn.commit()

In [45]:
import re
with open('Addresses.csv', 'r', encoding='utf-8-sig') as f:
    reader = csv.DictReader(f)
    addresses = list(reader)

    for row in addresses:
        # print(row)
        if row['Longitude'] == '' or row['Latitude'] == '':
            # print('Skipping')

            if row['Address to Validate'].startswith('Lat:'):
                reOut = re.findall(r'Lat:(-?[0-9]\d*(\.\d+)?) Long:(-?[0-9]\d*(\.\d+)?)', row['Address to Validate'])
                # print(reOut[0][0], reOut[0][2])
                c.execute("INSERT INTO addresses VALUES (:Address, :Longitude, :Latitude)", (row['Address to Validate'], float(reOut[0][2])/10000, float(reOut[0][0])/10000))
            continue
        c.execute("INSERT INTO addresses VALUES (:Address, :Longitude, :Latitude)", (row['Address to Validate'], float(row['Longitude']), float(row['Latitude'])))

conn.commit()

-266500 1530593
-272654 1530002
-266538 1530612
-271590 1529798
-271591 1529799
-271591 1529798
-271589 1529799
-271590 1529799
-271591 1529800
-271589 1529800
-271622 1529834
-270427 1529755
-270424 1529757
-270425 1529755
-270758 1531422
-271564 1529757
-271774 1529148
-270426 1529755
-270425 1529754
-270426 1529756
-268246 1531456
-271563 1529757
-266442 1530553
-264167 1529057
-271560 1529754
-266504 1530599
-266492 1530574
-266492 1530575
-266503 1530598
-266492 1530576
-266500 1530592
-266501 1530596
-266504 1530598
-266493 1530577
-266488 1530576
-266497 1530611
-266501 1530595
-272654 1530003
-272653 1530002
-272662 1530003
-272661 1530002
-272669 1530012
-272654 1530001
-272657 1530000
-272655 1530003
-272594 1530872
-272657 1530018
-272658 1530018
-272656 1530001
-272653 1530003
-272662 1530002
-272663 1530012
-267653 1528682
-267648 1528679
-267651 1528679
-267652 1528682
-266497 1530608
-272655 1530002
-272593 1530871
-272592 1530871
-272655 1530001
-272662 1530001
-272589 

In [48]:
c.execute("DELETE FROM Addresses")
conn.commit()

OperationalError: near "FROM": syntax error